In [66]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [67]:
df = pd.read_csv('data/cannabis.csv')

In [68]:
df=df.dropna()

In [69]:
print(df.shape)
df

(2277, 6)


,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."
...,...,...,...,...,...,...
2346,Zeus-Og,hybrid,4.7,"Happy,Uplifted,Relaxed,Euphoric,Energetic","Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...
2347,Zkittlez,indica,4.6,"Relaxed,Happy,Euphoric,Uplifted,Sleepy","Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...
2348,Zombie-Kush,indica,5.0,"Relaxed,Sleepy,Talkative,Euphoric,Happy","Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...
2349,Zombie-Og,indica,4.4,"Relaxed,Sleepy,Euphoric,Happy,Hungry","Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...


In [70]:
df['combined'] = df["Effects"] + ',' + df["Flavor"] + df['Description']
df["combined"][0]

'Creative,Energetic,Tingly,Euphoric,Relaxed,Earthy,Sweet,Citrus$100 OG is a 50/50 hybrid strain that packs a strong punch. The name supposedly refers to both its strength and high price when it first started showing up in Hollywood. As a plant, $100 OG tends to produce large dark green buds with few stems. Users report a strong body effect of an indica for pain relief with the more alert, cerebral feeling thanks to its sativa side.'

In [71]:
df.head()

,Strain,Type,Rating,Effects,Flavor,Description,combined
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"Creative,Energetic,Tingly,Euphoric,Relaxed,Ear..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"Relaxed,Aroused,Creative,Happy,Energetic,Flowe..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"Uplifted,Happy,Relaxed,Energetic,Creative,Spic..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"Tingly,Creative,Hungry,Relaxed,Uplifted,Aprico..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","Happy,Relaxed,Euphoric,Uplifted,Talkative,Citr..."


In [74]:
import spacy
from spacy.tokenizer import Tokenizer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("en_core_web_lg")
df.head()

# The Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Make the tokens for description
combined_tokens = []
for txt in tokenizer.pipe(df['combined'], batch_size=500):
    txt_tokens = [token.text for token in txt]
    description_tokens.append(txt_tokens)
df['combined_tokens'] = description_tokens
print(df['combined_tokens'].head())

0    [Creative,Energetic,Tingly,Euphoric,Relaxed,Ea...
1    [Relaxed,Aroused,Creative,Happy,Energetic,Flow...
2    [Uplifted,Happy,Relaxed,Energetic,Creative,Spi...
3    [Tingly,Creative,Hungry,Relaxed,Uplifted,Apric...
4    [Happy,Relaxed,Euphoric,Uplifted,Talkative,Cit...
Name: combined_tokens, dtype: object


In [75]:
df.head()

,Strain,Type,Rating,Effects,Flavor,Description,combined,combined_tokens
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"Creative,Energetic,Tingly,Euphoric,Relaxed,Ear...","[Creative,Energetic,Tingly,Euphoric,Relaxed,Ea..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"Relaxed,Aroused,Creative,Happy,Energetic,Flowe...","[Relaxed,Aroused,Creative,Happy,Energetic,Flow..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"Uplifted,Happy,Relaxed,Energetic,Creative,Spic...","[Uplifted,Happy,Relaxed,Energetic,Creative,Spi..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"Tingly,Creative,Hungry,Relaxed,Uplifted,Aprico...","[Tingly,Creative,Hungry,Relaxed,Uplifted,Apric..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","Happy,Relaxed,Euphoric,Uplifted,Talkative,Citr...","[Happy,Relaxed,Euphoric,Uplifted,Talkative,Cit..."


In [94]:
df['combined_tokens'][0]

['Creative,Energetic,Tingly,Euphoric,Relaxed,Earthy,Sweet,Citrus$100',
 'OG',
 'is',
 'a',
 '50/50',
 'hybrid',
 'strain',
 'that',
 'packs',
 'a',
 'strong',
 'punch.',
 'The',
 'name',
 'supposedly',
 'refers',
 'to',
 'both',
 'its',
 'strength',
 'and',
 'high',
 'price',
 'when',
 'it',
 'first',
 'started',
 'showing',
 'up',
 'in',
 'Hollywood.',
 'As',
 'a',
 'plant,',
 '$100',
 'OG',
 'tends',
 'to',
 'produce',
 'large',
 'dark',
 'green',
 'buds',
 'with',
 'few',
 'stems.',
 'Users',
 'report',
 'a',
 'strong',
 'body',
 'effect',
 'of',
 'an',
 'indica',
 'for',
 'pain',
 'relief',
 'with',
 'the',
 'more',
 'alert,',
 'cerebral',
 'feeling',
 'thanks',
 'to',
 'its',
 'sativa',
 'side.']

In [76]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [77]:

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words = 'english',
                       #ngram_range = (1,2),
                       max_features = 5000)

# Create a vocabulary and tf-idf score per document
dtm = tfidf.fit_transform(df['combined'])
                         

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
nn.fit(dtm)


# View Feature Matrix as DataFrame
print(dtm.shape)
dtm.head()

(2277, 5000)


,09,10,100,11,12,13,14,15,16,17,...,zen,zest,zestful,zesty,zeta,zion,zkittlez,zombie,zone,zoning
0,0.0,0.0,0.41632,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.00000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.00000,0.0,0.0,0.3043,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.00000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
ideal = ["""
Creative,Uplifted,Tingly,Euphoric,Relaxed, Giggly
"""]

In [79]:
# Query the ideal descprition
new = tfidf.transform(ideal)
new

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [80]:
nn.kneighbors(new.todense())

(array([[1.0705713 , 1.17907498, 1.20176509, 1.20775674]]),
 array([[1681,  499,  813,  643]]))

In [81]:
# most ideal description,strain and flavor
df['Description'][1681]

'Purple Cream has been a favorite of anxiety sufferers for over a decade. Great for nighttime use, this indica is mellow and calming. Purple Cream produces a moderate body-heavy sensation, relaxing your muscles and easing stress. The effects of this strain are quite sedative and can last around two hours. Like other purples, Purple Cream’s aroma is sweet, floral, and earthy.'

In [83]:
df['Strain'][1681]

'Purple-Cream'

In [84]:
df['Flavor'][1681]

'Sweet,Pine,Earthy'

In [85]:
df['Effects'][1681]

'Happy,Relaxed,Sleepy,Hungry,Giggly'

In [86]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename = 'model.pkl2'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(nn, pickled_model)
pickled_model.close() # Close the pickle instances

In [89]:
# Loading the saved model
model_pkl2 = open(pickle_filename, 'rb')
NN_model2 = pickle.load(model_pkl2)
print ("Loaded model :: ", NN_model2)  # print to verify

Loaded model ::  NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)


In [100]:
# Dump the trained classifier (tfidf)  with Pickle
pickle_filename_1 = 'tfidf.pkl2'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(tfidf, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

In [101]:
# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
tfidf_model = pickle.load(model_pkl_1)
print ("Loaded model :: ", tfidf_model)  # print to verify

Loaded model ::  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [90]:
ideal2 = ["happy,Sleepy,Apricot,Citrus,Grapefruit"]

In [102]:
new2 = tfidf_model.transform(ideal2)
new2

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [103]:
NN_model2.kneighbors(new2.todense())

(array([[1.09938463, 1.16838824, 1.18974831, 1.19398648]]),
 array([[2013,  912,  915, 1745]]))

In [104]:
df['Strain'][2013]

'Strawberry-Lemonade'

In [115]:
import json
def recommend(user_input):
    temp_df = NN_model.kneighbors(tfidf_model.transform([user_input]).todense())[1]
    

    #print(temp_df)
    
    for i in range(4):
        info = df.loc[temp_df[0][i]]['Strain']
        info_effects = df.loc[temp_df[0][i]]['Effects']
        info_flavor = df.loc[temp_df[0][i]]['Flavor']
        info_description = df.loc[temp_df[0][i]]['Description']
        info_rating = df.loc[temp_df[0][i]]['Rating']
        
        print(json.dumps(info))
        print(json.dumps(info_effects))
        print(json.dumps(info_flavor))
        print(json.dumps(info_description))
        print(json.dumps(info_rating))
        
        #return json.dumps(info)  #for engineeers, the return does not work in jupyter lab.  Should work in vsCode.
        #return json.dumps(info_effects)
        #return json.dumps(info_flavor)
        #return json.dump(info_description)
        #return json.dumps(info_rating)

In [116]:
recommend('Creative,Energetic,Tingly,Euphoric,Relaxed')

"Pineapple-Super-Silver-Haze"
"Happy,Euphoric,Energetic,Focused,Tingly"
"Pineapple,Sweet,Earthy"
"Pineapple Super Silver Haze from Fire Bros. is a sativa strain that modifies the widely cherished Super Silver Haze with a Pineapple hybrid strain. This second parent is thought to be either Pineapple Express or Pineapple, a phenotype of Ed Rosenthal Super Bud. The Haze genetics in this strain come through in both flavor and effect, as Pineapple Super Silver Haze delivers a high-flying cerebral buzz alongside a spicy, zesty flavor. Its aroma is more of a tropical medley of candied mango, pineapple, and oranges. Like a cup of coffee, this sativa is a perfect pick-me-up with motivating, talkative, and creative effects and minimal heaviness in the body."
4.7
"Pineapple-Thai"
"Relaxed,Creative,Sleepy,Happy,Focused"
"Pineapple,Citrus,Sweet"
"Pineapple Thai is a strain that sets itself apart from the rest with a high 5% CBD content. This flower\u2019s attractiveness comes from its ability to adm